In [ ]:
# installing all the necessary dependencies
import os, re
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, FunctionTransformer
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    HistGradientBoostingRegressor,
    StackingRegressor
)
from sklearn.linear_model import Ridge
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_absolute_error
import joblib

In [ ]:
# ---------- 1) LOAD & BASIC CLEAN ----------
csv_path = "PRSA_data_2010.1.1-2014.12.31.csv" 
assert os.path.exists(csv_path), f"CSV file not found at {csv_path}"

df = pd.read_csv(csv_path)
print("Original columns:", list(df.columns)[:20])
print(df.shape)
df.head()

In [ ]:
# Normalize column names to friendly lowercase identifiers
def clean_col(c):
    c = str(c).strip().lower()
    # unify pm2.5 -> pm25
    c = c.replace("pm2.5", "pm25")
    # replace non-alphanum with underscore
    c = re.sub(r'[^0-9a-z]+', '_', c)
    c = c.strip('_')
    return c

df.columns = [clean_col(c) for c in df.columns]
print("Normalized columns:", list(df.columns))

In [ ]:
# ---------- 2) BUILD DATETIME ----------
# UCI has year, month, day, hour columns; construct a single datetime column
if set(['year', 'month', 'day', 'hour']).issubset(df.columns):
    df['datetime'] = pd.to_datetime(df[['year','month','day','hour']])
    df = df.sort_values('datetime').reset_index(drop=True)
    df.set_index('datetime', inplace=False)
else:
    # If dataset already has a timestamp column - try 'time' or 'date' detection
    possible_ts = [c for c in df.columns if 'date' in c or 'time' in c or 'datetime' in c]
    if possible_ts:
        df['datetime'] = pd.to_datetime(df[possible_ts[0]])
    else:
        raise ValueError("Couldn't find (year,month,day,hour) nor timestamp column. Please add a datetime column.")

df = df.sort_values('datetime').reset_index(drop=True)
df['hour'] = df['datetime'].dt.hour
df['dayofweek'] = df['datetime'].dt.dayofweek
df['month'] = df['datetime'].dt.month
df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)

In [ ]:
# ---------- 3) CYCLICAL ENCODING ----------
df['hour_sin'] = np.sin(2 * np.pi * df['hour']/24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour']/24)
df['dow_sin'] = np.sin(2 * np.pi * df['dayofweek']/7)
df['dow_cos'] = np.cos(2 * np.pi * df['dayofweek']/7)

In [ ]:
# ---------- 4) RENAME / INTERPRET COLUMNS ----------
# Typical UCI fields: pm25, dewp, temp, pres, cbwd (wind direction categorical), iws (wind speed), is, ir
# Map common names:
col_map = {}
if 'pm25' in df.columns:
    col_map['pm25'] = 'pm25'
if 'dewp' in df.columns:
    col_map['dewp'] = 'dewpoint'
if 'temp' in df.columns:
    col_map['temp'] = 'temp'
if 'pres' in df.columns:
    col_map['pres'] = 'pressure'
if 'iws' in df.columns:
    col_map['iws'] = 'wind_speed'
if 'cbwd' in df.columns:
    col_map['cbwd'] = 'cbwd'   # categorical wind-dir
    
df = df.rename(columns=col_map)
print("After rename, sample columns:", list(df.columns))

In [ ]:
# ---------- 5) LAGS & ROLLING FEATURES ----------
# We'll create lag features from pm25 (past values only)
target_col = 'pm25'
lags = [1, 3, 6, 12, 24]   # hours
for lag in lags:
    df[f'pm25_lag_{lag}'] = df[target_col].shift(lag)

# rolling stats (use shifted series so we never use 'future' info)
df['pm25_roll_mean_6h'] = df[target_col].shift(1).rolling(window=6, min_periods=1).mean()
df['pm25_roll_std_6h']  = df[target_col].shift(1).rolling(window=6, min_periods=1).std().fillna(0)

# slope / trend over last 3 hours (simple linear slope)
def rolling_slope(series, window=3):
    # compute slope using polyfit (fast enough)
    arr = series.values
    out = np.full_like(arr, np.nan, dtype=float)
    for i in range(window-1, len(arr)):
        x = np.arange(window)
        y = arr[i-window+1:i+1]
        if np.any(np.isnan(y)):
            out[i] = np.nan
        else:
            out[i] = np.polyfit(x, y, 1)[0]
    return pd.Series(out, index=series.index)

df['pm25_slope_3h'] = rolling_slope(df[target_col].shift(1), window=3)

In [ ]:
# ---------- 6) WIND FEATURES ----------
# UCI has cbwd categorical (NE, NW, SE, cv). We'll one-hot encode cbwd in the pipeline.
# Optional: map cbwd -> approximate u/v (only if you want numeric wind vector)
wind_map = {'NE':45, 'NW':315, 'SE':135, 'cv':np.nan}  # 'cv' often means "calm / variable"
if 'cbwd' in df.columns:
    df['cbwd'] = df['cbwd'].astype(str).replace('nan', np.nan)

# if numeric wind_speed exists (iws), we can optionally estimate u/v by mapping cbwd to angle
if 'wind_speed' in df.columns and 'cbwd' in df.columns:
    ang = df['cbwd'].map(wind_map)
    df['wind_u'] = df['wind_speed'] * np.cos(np.deg2rad(ang))
    df['wind_v'] = df['wind_speed'] * np.sin(np.deg2rad(ang))

In [ ]:
# ---------- 7) TRAFFIC & NEIGHBORS ----------
# UCI does NOT include traffic_count or neighbor stations. If you have external traffic or other stations,
# join them on datetime BEFORE running the pipeline.
if 'traffic_count' not in df.columns:
    print("Note: no 'traffic_count' column found. Traffic-based features will be skipped unless you merge traffic data.")
else:
    df['traffic_roll3'] = df['traffic_count'].shift(1).rolling(3).mean()
    df['traffic_roll6'] = df['traffic_count'].shift(1).rolling(6).mean()

In [ ]:
# ---------- 8) MISSING TARGET HANDLING ----------
# For supervised regression we need rows where current pm25 (target) is present.
initial_len = len(df)
df = df[~df[target_col].isna()].copy()   # drop rows where target is missing
print(f"Dropped {initial_len - len(df)} rows with missing target (pm25).")

# drop rows where any of the lag features are still NaN (start of series)
lag_cols = [f'pm25_lag_{l}' for l in lags] + ['pm25_roll_mean_6h','pm25_roll_std_6h','pm25_slope_3h']
df = df.dropna(subset=lag_cols, how='any').reset_index(drop=True)
print("After dropping lag-NaN rows:", df.shape)

In [ ]:
# ---------- 9) DEFINE FEATURE LISTS ----------
num_cols = []
for c in ['dewpoint','temp','pressure','wind_speed','wind_u','wind_v']:
    if c in df.columns:
        num_cols.append(c)

# add pm2.5 lags & rolling stats
num_cols += [c for c in lag_cols if c in df.columns]

# cyclical numeric encodings
num_cols += ['hour_sin', 'hour_cos', 'dow_sin', 'dow_cos']

cat_cols = []
for c in ['month','is_weekend','cbwd']:   # cbwd categorical wind-dir
    if c in df.columns:
        cat_cols.append(c)

print("Numeric features (example):", num_cols[:20])
print("Categorical features (example):", cat_cols)

In [ ]:
# PM2.5 over time
import matplotlib.pyplot as plt

plt.figure(figsize=(15,5))
plt.plot(df['datetime'], df['pm25'], alpha=0.7)
plt.title("PM2.5 Concentration Over Time")
plt.xlabel("Date")
plt.ylabel("PM2.5 (µg/m³)")
plt.show()

In [ ]:
# Monthly average trend
df['month_name'] = df['datetime'].dt.to_period('M')
monthly_mean = df.groupby('month_name')['pm25'].mean()

plt.figure(figsize=(12,5))
monthly_mean.plot()
plt.title("Monthly Average PM2.5")
plt.ylabel("PM2.5 (µg/m³)")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Distribution of PM2.5
plt.figure(figsize=(6,4))
plt.hist(df['pm25'].dropna(), bins=50, alpha=0.7)
plt.title("Distribution of PM2.5")
plt.xlabel("PM2.5 (µg/m³)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
# Correlation heatmap
import seaborn as sns

plt.figure(figsize=(10,6))
sns.heatmap(df[['pm25','temp','dewpoint','pressure','wind_speed']].corr(),
            annot=True, cmap='coolwarm')
plt.title("Correlation Heatmap (PM2.5 vs Weather)")
plt.show()

In [ ]:
# Lag effect check (scatter of lag vs PM2.5)
plt.figure(figsize=(6,6))
plt.scatter(df['pm25_lag_1'], df['pm25'], alpha=0.3)
plt.title("Lag-1 PM2.5 vs Current PM2.5")
plt.xlabel("PM2.5 (t-1h)")
plt.ylabel("PM2.5 (t)")
plt.show()

In [ ]:
# ---------- 10) PREPROCESSOR (sklearn pipeline) ----------
num_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('scale', RobustScaler()),
    ('pca', PCA(n_components=0.95, svd_solver='full'))   # optional -- keeps 95% variance
])

cat_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', num_pipe, num_cols),
    ('cat', cat_pipe, cat_cols)
], remainder='drop', verbose_feature_names_out=False)

## Week 2

In [ ]:
import os
import sklearn, math
from packaging import version
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.inspection import permutation_importance
import joblib
import numpy as np

In [ ]:
os.makedirs("plots", exist_ok=True)

In [ ]:
# sklearn-version-safe OneHotEncoder argument
ohe_kwargs = {}
if version.parse(sklearn.__version__) >= version.parse("1.2"):
    ohe_kwargs['sparse_output'] = False
else:
    ohe_kwargs['sparse'] = False

In [ ]:
# Build a preprocessor WITHOUT PCA (keeps interpretability for now)
num_pipe_no_pca = Pipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('scale', RobustScaler()),
    # no PCA here for model interpretability
])
cat_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', **ohe_kwargs))
])

preprocessor_no_pca = ColumnTransformer(
    transformers=[
        ('num', num_pipe_no_pca, num_cols),
        ('cat', cat_pipe, cat_cols)
    ],
    remainder='drop', verbose_feature_names_out=False
)

In [ ]:
# Build selector + base models + stacking ensemble 
selector = SelectFromModel(RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=42),
                           threshold='median')

hgb = HistGradientBoostingRegressor(max_iter=300, learning_rate=0.05)
rf = RandomForestRegressor(n_estimators=400, n_jobs=-1, random_state=42)
et = ExtraTreesRegressor(n_estimators=400, n_jobs=-1, random_state=42)

stack = StackingRegressor(
    estimators=[('hgb', hgb), ('rf', rf), ('et', et)],
    final_estimator=Ridge(alpha=1.0),
    passthrough=True,
    n_jobs=-1
)

model_pipe = Pipeline([
    ('pre', preprocessor_no_pca),
    ('select', selector),
    ('model', stack)
])

In [ ]:
# HOLDOUT SPLIT: last 10% as time-based holdout
n_total = len(df)
holdout_frac = 0.10
holdout_size = max(1, int(n_total * holdout_frac))
train_df = df.iloc[:-holdout_size].reset_index(drop=True)
hold_df  = df.iloc[-holdout_size:].reset_index(drop=True)

X_train = train_df[num_cols + cat_cols]
y_train = train_df[target_col]
X_hold  = hold_df[num_cols + cat_cols]
y_hold  = hold_df[target_col]

print(f"Train rows: {len(X_train)}, Holdout rows: {len(X_hold)}")

In [ ]:
# TimeSeries CV on training partition
tscv = TimeSeriesSplit(n_splits=5)
cv_scores = cross_val_score(model_pipe, X_train, y_train,
                            cv=tscv, scoring='neg_mean_absolute_error', n_jobs=-1)
print("CV MAE (train portion): {:.3f} ± {:.3f}".format(-cv_scores.mean(), cv_scores.std()))

In [ ]:
# Fit final model on full training partition
print("Fitting final pipeline on full training partition (this may take time)...")
model_pipe.fit(X_train, y_train)

In [ ]:
# Evaluate on holdout (true "future")
y_hold_pred = model_pipe.predict(X_hold)
mae_hold = mean_absolute_error(y_hold, y_hold_pred)
rmse_hold = math.sqrt(mean_squared_error(y_hold, y_hold_pred))
r2_hold = r2_score(y_hold, y_hold_pred)
print(f"Holdout MAE: {mae_hold:.3f}, RMSE: {rmse_hold:.3f}, R2: {r2_hold:.3f}")